## AP Ranking in Brazil using Pin Move Refinement Data

In [ ]:
from __future__ import division
import ipyleaflet
import lime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import shap
import sklearn
import sklearn.inspection
import warnings
import xgboost as xgb

from eli5 import show_prediction, explain_prediction
from ipyleaflet import Map, Heatmap, Marker, CircleMarker, MarkerCluster, basemaps, basemap_to_tiles, AwesomeIcon, LegendControl
from IPython.display import Image, display_html
from ipywidgets import HTML, IntSlider, Output
from lime import lime_tabular
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, log_loss, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

random_seed = 42
warnings.filterwarnings('ignore')

### Features

In [ ]:
numerical_same_value_features = [
    'request_gps_and_anchor_point_distance',
    'request_gps_naive_snap_snapped_distance',
    'pickup_anchor_naive_snap_snapped_distance'
]

numerical_different_features = [
    'num_trips',
    'arrived_cancel',
    'arrived_contacts',
    'avg_nple',
    'nple_p95',
    'loops_around', 
    'max_circle',
    'overshoot_distance',
    'distance_to_request_gps',
    'distance_to_anchorpoint',
    'distance_to_pickup_anchor_naive_snap',
    'distance_to_request_gps_naive_snap'
]

numerical_features = numerical_same_value_features + numerical_different_features

categorical_features = [    
    'pickup_anchor_confidence',
    'provider',
    'product_type_name',
    'time_bucket',
    'city_id',
    'pickup_waypoint_location_source',
    'request_gps_naive_snap_traffic_direction',
    'request_gps_naive_snap_road_usage',
    'request_gps_naive_snap_road_side',
    'request_gps_naive_snap_road_class',
    'pickup_anchor_naive_snap_traffic_direction',
    'pickup_anchor_naive_snap_road_usage',
    'pickup_anchor_naive_snap_road_side',
    'pickup_anchor_naive_snap_road_class'
]

### Data

In [ ]:
DATA_DIR = '<DATASET_DIR>'
df_raw = pd.read_csv(os.path.join(DATA_DIR, '<data>.csv'))

In [ ]:
print(f"Number of trips in raw data: {df_raw.shape[0]}")

df_raw.columns = [x.split('.')[1] if '.' in (x) else x for x in df_raw.columns]

# drop duplicates
df_raw = df_raw.drop_duplicates(subset=['trip_uuid'])
print(f"Number of trips after dropping duplicates: {df_raw.shape[0]}")

# apply first distance to last filter
df_raw = df_raw.loc[df_raw['first_distance_to_pickup'] >= 10]
print(f"Number of trips after applying first_distance_to_pickup filter: {df_raw.shape[0]}")

# apply PLE filter
df_raw = df_raw.loc[df_raw['pickup_location_error'] <= 50]
print(f"Number of trips after applying PLE filter: {df_raw.shape[0]}")

# drop NULL values
for col in numerical_different_features + ['hex_13']:
    for prefix in ['first', 'last', 'begintrip', 'pickup']:
        df_raw = df_raw.loc[pd.notnull(df_raw[f'{prefix}_' + col]) & (df_raw[f'{prefix}_' + col] != '\\N') & (df_raw[f'{prefix}_' + col] != float('inf')) & (df_raw[f'{prefix}_' + col] != -float('inf'))]
print(f"Number of trips after dropping null values: {df_raw.shape[0]}")

In [ ]:
def generate_features(df, preferred_colname, unpreferred_colname):
    for feature in numerical_same_value_features + categorical_features + ['datestr', 'trip_uuid']:
        if feature in categorical_features:
            from sklearn import preprocessing
            le = preprocessing.LabelEncoder()
            le.fit(df[feature])
            df[feature] = le.transform(df[feature])
        df[f'{preferred_colname}_{feature}'] = df[f'{unpreferred_colname}_{feature}'] = df[feature]
        
    # take log of the num_trips column
    df[f'{preferred_colname}_num_trips'] = np.log(df[f'{preferred_colname}_num_trips'])
    df[f'{unpreferred_colname}_num_trips'] = np.log(df[f'{unpreferred_colname}_num_trips'])
    
    return df

def filter_data(df_input, preferred_colname, unpreferred_colname, feature_list, filter_p95=False, filter_p5=False, filter_p99=False):
    df_input = df_input.copy()
    print(f"Number of examples before filtering: {df_input.shape[0]}")
    
    # drop NULL values
    for col in feature_list:
        df_input = df_input.loc[pd.notnull(df_input[f'{unpreferred_colname}_' + col]) & (df_input[f'{unpreferred_colname}_' + col] != '\\N') & (df_input[f'{unpreferred_colname}_' + col] != float('inf')) & (df_input[f'{unpreferred_colname}_' + col] != -float('inf'))]
        df_input = df_input.loc[pd.notnull(df_input[f'{preferred_colname}_' + col]) & (df_input[f'{preferred_colname}_' + col] != '\\N') & (df_input[f'{preferred_colname}_' + col] != float('inf')) & (df_input[f'{preferred_colname}_' + col] != -float('inf'))]
        try:
            df_input[f'{unpreferred_colname}_' + col] = df_input[f'{unpreferred_colname}_' + col].astype(float)
            df_input[f'{preferred_colname}_' + col] = df_input[f'{preferred_colname}_' + col].astype(float)
        except:
            continue
    print(f"Number of examples after dropping null values: {df_input.shape[0]}")

    # clean data
    df_input = df_input[
        (df_input[f'first_distance_to_{preferred_colname}'] < 200)
        & (df_input[f'{preferred_colname}_distance_to_request_gps'] <= 200)
        & (df_input[f'{unpreferred_colname}_distance_to_request_gps'] <= 200)
    ]
    print(f'Number of examples in cleaned data: {df_input.shape[0]}')
    
    if filter_p99:
        for col in feature_list:
            if df_input[f'{unpreferred_colname}_{col}'].dtype == 'float64':
                df_input = df_input.loc[df_input[f'{unpreferred_colname}_{col}'] <= np.percentile(df_input[f'{unpreferred_colname}_{col}'], 99)]
                df_input = df_input.loc[df_input[f'{preferred_colname}_{col}'] <= np.percentile(df_input[f'{preferred_colname}_{col}'], 99)]
        print(f'Number of examples after applying p99 filter: {df_input.shape[0]}')
        
    if filter_p95:
        for col in feature_list:
            if df_input[f'{unpreferred_colname}_{col}'].dtype == 'float64':
                df_input = df_input.loc[df_input[f'{unpreferred_colname}_{col}'] <= np.percentile(df_input[f'{unpreferred_colname}_{col}'], 95)]
                df_input = df_input.loc[df_input[f'{preferred_colname}_{col}'] <= np.percentile(df_input[f'{preferred_colname}_{col}'], 95)]
        print(f'Number of examples after applying p95 filter: {df_input.shape[0]}')
        
    if filter_p5:
        for col in feature_list:
            if df_input[f'{unpreferred_colname}_{col}'].dtype == 'float64':
                df_input = df_input.loc[df_input[f'{unpreferred_colname}_{col}'] >= np.percentile(df_input[f'{unpreferred_colname}_{col}'], 5)]
                df_input = df_input.loc[df_input[f'{preferred_colname}_{col}'] >= np.percentile(df_input[f'{preferred_colname}_{col}'], 5)]
        print(f'Number of examples after applying p5 filter: {df_input.shape[0]}')

    return df_input

def generate_training_data(df, preferred_colname, unpreferred_colname, features_list):
    df = df.copy()
    
    features_list_unpreferred, features_list_preferred = [], []
    for col in features_list + ['datestr', 'trip_uuid']:
        features_list_unpreferred.append(f'{unpreferred_colname}_{col}')
        features_list_preferred.append(f'{preferred_colname}_{col}')
    
    df_unpreferred = df[features_list_unpreferred]
    df_unpreferred.columns = [col.split(f'{unpreferred_colname}_')[1] for col in features_list_unpreferred]
    df_unpreferred['y'] = 0
    
    df_preferred = df[features_list_preferred]
    df_preferred.columns = [col[len(f'{preferred_colname}_'):] for col in features_list_preferred]
    df_preferred['y'] = 1

    df = pd.concat([df_unpreferred, df_preferred], axis=0)
    df = df.sort_values(by=['datestr'])
    
    return df

### Models

In [ ]:
def logistic_regression(df, feature_list):
    """
    Train logistic regression model and return its parameters.
    
    :param df: transformed dataframe including both training and test set
    :param feature_list: list of features to be put into the model
    :return:
        params dictionary of the logistic regression;
        thresholds dictionary to handle large values of each feature.
    """
    thresholds = dict(df[[c for c in feature_list if c != 'distancetotarget']].describe().loc['max'])

    X = np.array(df[feature_list])
    y = np.array(df['y'])

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)
    n = X.shape[0]
    X_train, X_test, y_train, y_test = X[:int(0.8 * n)], X[int(0.8 * n):], y[:int(0.8 * n)], y[int(0.8 * n):]
    
    # normalize data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    print(f"Normalization means: {scaler.mean_}")
    
    # model
    clf = LogisticRegression(random_state=random_seed, max_iter=500).fit(X_train, y_train)
    print('Accuracy of training set =', clf.score(X_train, y_train))
    print('Accuracy of test set =', clf.score(X_test, y_test))
    params = dict(zip(feature_list + ['intercept'], list(clf.coef_[0]) + [clf.intercept_[0]]))
    print('Parameters: ', params)

    # classification report
    y_test_pred = clf.predict(X_test)
    y_test_pred_proba = clf.predict_proba(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_test_pred))
    print("Confusion Matrix:", confusion_matrix(y_test, y_test_pred))
    print("Log loss =", log_loss(y_test, y_test_pred_proba))
    print("AUC-ROC =", roc_auc_score(y_test, y_test_pred_proba[:, 1]))

    return params, thresholds, clf, X_train, y_train

In [ ]:
def svm(df, feature_list):
    X, y = np.array(df[feature_list]), np.array(df['y'])

    n = X.shape[0]
    X_train, X_test, y_train, y_test = X[:int(0.8 * n)], X[int(0.8 * n):], y[:int(0.8 * n)], y[int(0.8 * n):]
    
    # normalize data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # model
    clf = sklearn.svm.SVC(random_state=random_seed, max_iter=500, probability=False, kernel='rbf').fit(X_train, y_train)
    print('Accuracy of training set =', clf.score(X_train, y_train))
    print('Accuracy of test set =', clf.score(X_test, y_test))

    # classification report
    y_test_pred = clf.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_test_pred))
    print("Confusion Matrix:", confusion_matrix(y_test, y_test_pred))

    return clf

In [ ]:
def xgboost(df, feature_list, monotone_constraints=None, enable_categorical=False):
    X, y = np.array(df[feature_list]), np.array(df['y'])
    n = X.shape[0]
    X_train, X_test, y_train, y_test = X[:int(0.8 * n)], X[int(0.8 * n):], y[:int(0.8 * n)], y[int(0.8 * n):]
    
    # model
    clf = xgb.XGBClassifier(tree_method='hist', n_jobs=1, monotone_constraints=monotone_constraints, 
                            enable_categorical=enable_categorical).fit(X_train, y_train)
    print('Accuracy of training set =', clf.score(X_train, y_train))
    print('Accuracy of test set =', clf.score(X_test, y_test))

    # classification report
    y_test_pred = clf.predict(X_test)
    y_test_pred_proba = clf.predict_proba(X_test)
    print("Classification Report:", classification_report(y_test, y_test_pred))
    print("Confusion Matrix:", confusion_matrix(y_test, y_test_pred))
    print("Log loss =", log_loss(y_test, y_test_pred_proba))
    print("AUC-ROC =", roc_auc_score(y_test, y_test_pred_proba[:, 1]))
    
    return clf

In [ ]:
def simple_pairwise_rank(df, feature_list, unpreferred_colname, preferred_colname):
    df = df.sort_values(by=['datestr'])
    unpreferred = df[[f'{unpreferred_colname}_{feature}' for feature in feature_list]].to_numpy()
    preferred = df[[f'{preferred_colname}_{feature}' for feature in feature_list]].to_numpy()
    X, y = preferred - unpreferred, np.ones(preferred.shape[0])
    X[::2] *= -1; y[::2] *= -1
    
    n = X.shape[0]
    X_train, X_test, y_train, y_test = X[:int(0.8 * n)], X[int(0.8 * n):], y[:int(0.8 * n)], y[int(0.8 * n):]
    
    # normalize data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # model
    clf = LogisticRegression(random_state=random_seed, max_iter=500).fit(X_train, y_train)
    print('Accuracy of training set =', clf.score(X_train, y_train))
    print('Accuracy of test set =', clf.score(X_test, y_test))
    params = dict(zip(train_features_list + ['intercept'], list(clf.coef_[0]) + [clf.intercept_[0]]))
    print('Parameters: ', params)

    # classification report
    y_test_pred = clf.predict(X_test)
    y_test_pred_proba = clf.predict_proba(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_test_pred))
    print("Confusion Matrix:", confusion_matrix(y_test, y_test_pred))
    print("Log loss =", log_loss(y_test, y_test_pred_proba))
    print("AUC-ROC =", roc_auc_score(y_test, y_test_pred_proba[:, 1]))
    
    return params, clf

In [ ]:
def xgbranker(df, feature_list, unpreferred_colname, preferred_colname, monotone_constraints=None, enable_categorical=False):
    df = df.sort_values(by=['datestr'])
    ngroups = df.shape[0]
    unpreferred = df[[f'{unpreferred_colname}_{feature}' for feature in feature_list]].to_numpy()
    preferred = df[[f'{preferred_colname}_{feature}' for feature in feature_list]].to_numpy()
    X = np.empty((2 * ngroups, len(feature_list)), dtype=np.float64)
    X[0::2] = unpreferred
    X[1::2] = preferred
    y = np.array([0,1] * ngroups)
    
    ntrain = int(0.8 * ngroups) * 2
    X_train, X_test, y_train, y_test = X[:ntrain], X[ntrain:], y[:ntrain], y[ntrain:]
    groups = 2 * np.ones(ntrain // 2)

    model = xgb.XGBRanker(
        tree_method='hist',
        booster='gbtree',
        objective='rank:pairwise',
        random_state=random_seed, 
        learning_rate=0.1,
        colsample_bytree=0.9,
        eta=0.05, 
        max_depth=7, 
        n_estimators=300, 
        subsample=0.75,
        monotone_constraints=monotone_constraints,
        enable_categorical=enable_categorical
    ).fit(X_train, y_train, group=groups)

    y_train_pred, y_test_pred = model.predict(X_train), model.predict(X_test)
    print('Accuracy of training set =', np.sum(y_train_pred[0::2] <= y_train_pred[1::2]) / (y_train_pred.shape[0] / 2))
    print('Accuracy of test set =', np.sum(y_test_pred[0::2] <= y_test_pred[1::2]) / (y_test_pred.shape[0] / 2))
    
    return model

## Experiments

In [ ]:
unpreferred_colname, preferred_colname = 'first', 'pickup'

df = df_raw.copy()
df = generate_features(df, preferred_colname, unpreferred_colname)

blocklisted_features = {
    'distance_to_request_gps_naive_snap',
    'request_gps_and_anchor_point_distance',
    'request_gps_naive_snap_snapped_distance',
    'pickup_anchor_naive_snap_snapped_distance',
    'loops_around', 
    'max_circle',
    'overshoot_distance',
    'distance_to_anchorpoint',
    'distance_to_pickup_anchor_naive_snap',
}
train_features_list = list(filter(lambda feature: feature not in blocklisted_features, numerical_features))

df = filter_data(df, preferred_colname, unpreferred_colname, feature_list=train_features_list, filter_p99=True)

data = generate_training_data(df, preferred_colname, unpreferred_colname, train_features_list)

#### Logistic Regression

In [ ]:
params, thresholds, model, X, y = logistic_regression(data, train_features_list)

# plot feature importance
names, values = [], []
for k, v in params.items():
    names.append(k); values.append(v)

rank = list(np.argsort(values)[::-1])
sns.set(rc = {'figure.figsize':(10,10)})
sns.set_theme(style="whitegrid")
ax = sns.barplot([names[i] for i in rank], [values[i] for i in rank], color="royalblue")
ax.set_xticklabels(ax.get_xticklabels(), rotation='vertical')
ax

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
pdp = sklearn.inspection.PartialDependenceDisplay.from_estimator(model, X[:5000], list(range(len(train_features_list))) + [(0,5)], feature_names=train_features_list, ax=ax)

#### XGBoost

In [ ]:
constraints = {'num_trips': 1,'arrived_cancel': -1,'arrived_contacts': -1,'avg_nple': -1,'nple_p95': -1,'distance_to_request_gps': -1,'distance_to_anchorpoint': -1}
model = xgboost(data, train_features_list, 
                monotone_constraints='(1,-1,-1,-1,-1,-1,-1,-1,-1)')

feature_important = list(model.get_booster().get_score(importance_type='gain').values())
rank = list(np.argsort(feature_important)[::-1])
ax = sns.barplot([train_features_list[i] for i in rank], [feature_important[i] for i in rank], color="royalblue")
ax.set_xticklabels(ax.get_xticklabels(), rotation='vertical')
ax

In [ ]:
model.get_booster().feature_names = train_features_list
fig, ax = plt.subplots(figsize=(240, 240))
xgb.plot_tree(model, num_trees=0, ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
sklearn.inspection.PartialDependenceDisplay.from_estimator(model, X[:10000], list(range(len(train_features_list))), feature_names=train_features_list, ax=ax)

#### Logistic Regression with Pairwise Transform

In [ ]:
params, model = simple_pairwise_rank(df, train_features_list, unpreferred_colname, preferred_colname)

# plot feature importance
names, values = [], []
for k, v in params.items():
    names.append(k); values.append(v)

rank = list(np.argsort(values)[::-1])
sns.set(rc = {'figure.figsize':(10,10)})
sns.set_theme(style="whitegrid")
ax = sns.barplot([names[i] for i in rank], [values[i] for i in rank], color="royalblue")
ax.set_xticklabels(ax.get_xticklabels(), rotation='vertical')
ax

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
sklearn.inspection.PartialDependenceDisplay.from_estimator(model, X, list(range(len(train_features_list))) + [(0,5)], feature_names=train_features_list, ax=ax)

#### LambdaMART

In [ ]:
model = xgbranker(df, train_features_list, unpreferred_colname, preferred_colname, '(1,-1,-1,-1,-1,-1)')

feature_important = list(model.get_booster().get_score(importance_type='gain').values())
rank = list(np.argsort(feature_important)[::-1])
ax = sns.barplot([train_features_list[i] for i in rank], [feature_important[i] for i in rank], color="royalblue")
ax.set_xticklabels(ax.get_xticklabels(), rotation='vertical')
ax

In [ ]:
model.get_booster().feature_names = train_features_list
fig, ax = plt.subplots(figsize=(240, 240))
xgb.plot_tree(model, num_trees=0, ax=ax)
# plt.gcf().set_size_inches(18.5, 20.5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
model._estimator_type = 'regressor'
sklearn.inspection.PartialDependenceDisplay.from_estimator(model, X, list(range(len(train_features_list))), feature_names=train_features_list, ax=ax)

### Model Explainability

In [ ]:
def get_base_map(center, zoom=20, height='400px', width='400px'):
    m = Map(basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik), center=center, zoom=zoom)
    m.layout.height, m.layout.width = height, width
    return m

def visualize_trip(df, idx):
    request_gps_lat, request_gps_lng = df.iloc[idx]['request_gps_lat'], df.iloc[idx]['request_gps_lng']
    m = get_base_map((request_gps_lat, request_gps_lng))
    
    m.add_layer(Marker(location=(request_gps_lat, request_gps_lng), icon=AwesomeIcon(name='male', marker_color='red'), draggable=False, opacity=1))
    m.add_layer(Marker(location=(df.iloc[idx]['pickup_anchor_lat'], df.iloc[idx]['pickup_anchor_lng']), icon=AwesomeIcon(name='anchor', marker_color='darkblue'), draggable=False, opacity=1.0))
    m.add_layer(Marker(location=(df.iloc[idx]['first_lat'], df.iloc[idx]['first_lng']), icon=AwesomeIcon(name='thumbs-down', marker_color='lightgray'), draggable=False, opacity=1))
    m.add_layer(Marker(location=(df.iloc[idx]['pickup_lat'], df.iloc[idx]['pickup_lng']), icon=AwesomeIcon(name='thumbs-up', marker_color='black'), draggable=False, opacity=1))
    
    display(m)

    trip_uuid = df.iloc[idx]['trip_uuid']
    examples = data[data['trip_uuid'] == trip_uuid]
    print('Unpreferred Access Point')
    display(show_prediction(model, examples[examples['y'] == 0][train_features_list].to_numpy().reshape((-1,)), show_feature_values=True, feature_names=train_features_list))
    print('Preferred Acces Point')
    display(show_prediction(model, examples[examples['y'] == 1][train_features_list].to_numpy().reshape((-1,)), show_feature_values=True, feature_names=train_features_list))

In [ ]:
visualize_trip(df, 500)

#### LIME

In [ ]:
def strip_html(htmldoc, strip_tags = ['html','meta','head','body'], outfile=None, verbose=False):    
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(htmldoc)
    
    for tag in strip_tags:
        rmtag = soup.find(tag)
        if rmtag is not None:
            rmtag.unwrap()
            if verbose: print(tag,'tags removed')
    
    stripped = soup.prettify()
    if outfile is not None:
        with open(outfile, 'w', encoding='utf-8') as f:
            f.write(stripped)
        if verbose: 
            print(f'file saved to: {outfile}')
    else:
        return stripped

limeparams = dict(
    training_data = X, 
    training_labels = y, 
    feature_names = train_features_list, 
    class_names = ['unpreferred','preferred']
)
lte = lime_tabular.LimeTabularExplainer(**limeparams)

In [ ]:
trip_uuid = '983c803d-9db2-4cc3-86fe-caedf57d617f'
examples = data[data['trip_uuid'] == trip_uuid]  
lte_expl = lte.explain_instance(examples[examples['y'] == 0][train_features_list].to_numpy().reshape((-1,)), model.predict_proba)
display_html(strip_html(lte_expl.as_html()), raw=True)

#### SHAP

In [ ]:
import shap
shap_xgbc = shap.TreeExplainer(model)
shapvals_xgbc = shap_xgbc.shap_values(X, y)

shap.initjs()
shap.force_plot(shap_xgbc.expected_value, shapvals_xgbc[0], features=X[0], link='logit')

In [ ]:
shap.summary_plot(shapvals_xgbc, X, feature_names=train_features_list, class_names=['unpreferred','preferred'])

In [ ]:
siv_xgbc = shap_xgbc.shap_interaction_values(X)
shap.summary_plot(siv_xgbc, X)

### Appendix
#### Manifold Learning

In [ ]:
from sklearn.manifold import TSNE

X, y = data[train_features_list], data['y'].to_numpy()

X_embedded = TSNE(n_components=2).fit_transform(X[:2500])

label = pd.DataFrame(['preferred' if label == 1 else 'unpreferred' for label in y[:2500]])
sns_data = pd.concat([pd.DataFrame(X_embedded), label], axis=1)
sns_data.columns = ['x', 'y', 'Label']

sns.set_theme()
sns.set(rc = {'figure.figsize':(10,10)})

sns.scatterplot(data=sns_data, x="x", y="y", hue="Label")

In [ ]:
feature_list = train_features_list
unpreferred = df[[f'{unpreferred_colname}_{feature}' for feature in feature_list]].to_numpy()
preferred = df[[f'{preferred_colname}_{feature}' for feature in feature_list]].to_numpy()
X, y = preferred - unpreferred, np.ones(preferred.shape[0])
X[::2] *= -1; y[::2] *= -1

X_embedded = TSNE(n_components=2).fit_transform(X[:2500])

label = pd.DataFrame(['1' if label == 1 else '-1' for label in y[:2500]])
sns_data = pd.concat([pd.DataFrame(X_embedded), label], axis=1)
sns_data.columns = ['x', 'y', 'Label']

sns.scatterplot(data=sns_data, x="x", y="y", hue="Label")